In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [22]:
flow = pd.read_csv('信息流数据_v1.csv')
flow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67122 entries, 0 to 67121
Data columns (total 27 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   推广位名称        67122 non-null  object 
 1   投放日期         67122 non-null  object 
 2   已投放天数        67122 non-null  int64  
 3   投放时段         49456 non-null  float64
 4   品牌名称         67122 non-null  object 
 5   视频编号         67122 non-null  object 
 6   账号           61527 non-null  object 
 7   消耗           67122 non-null  float64
 8   展示数          67122 non-null  int64  
 9   点击数          67122 non-null  int64  
 10  转化数          67122 non-null  int64  
 11  按钮button     67122 non-null  int64  
 12  调起店铺         67122 non-null  int64  
 13  播放数          67122 non-null  int64  
 14  有效播放数        67122 non-null  int64  
 15  WiFi播放占比(%)  67122 non-null  float64
 16  商品ID         67122 non-null  int64  
 17  页面PV         67122 non-null  int64  
 18  页面UV         67122 non-null  int64  
 19  收藏UV

In [23]:
# 删除日期列
flow.drop('投放日期',axis=1, inplace=True)

In [24]:
# 缺失值检查
pd.isnull(flow).sum()
pd.isna(flow).sum()

推广位名称              0
已投放天数              0
投放时段           17666
品牌名称               0
视频编号               0
账号              5595
消耗                 0
展示数                0
点击数                0
转化数                0
按钮button           0
调起店铺               0
播放数                0
有效播放数              0
WiFi播放占比(%)        0
商品ID               0
页面PV               0
页面UV               0
收藏UV               0
加入购物车UV            0
综合成交笔数             0
综合成交金额             0
直接成交订单笔数         326
直接成交订单金额           0
间接成交笔数             0
间接成交金额             0
dtype: int64

In [25]:
# 缺失值简单填充：分组填充投放时段和账号
flow['投放时段'] = flow.groupby("推广位名称").transform(lambda x: x.fillna(x.median()))
flow['账号'] = flow.groupby("推广位名称").transform(lambda x: x.fillna(x.mode()))

In [26]:
# 异常值处理
flow = flow.query('已投放天数>0').reset_index()
flow = flow.query('投放时段>0').reset_index() 
flow.drop(['level_0','index'],axis = 1,inplace = True)
flow = flow.query('消耗>0').reset_index() 
flow.drop(['index'],axis = 1,inplace = True)

In [27]:
# one-hot ecoding(此处先one-hot的原因是对直接成交单数进行填充时需要其它变量作为feature输入)
推广位名称_dum=pd.get_dummies(flow['推广位名称'])
品牌名称_dum=pd.get_dummies(flow['品牌名称'])
视频编号_dum=pd.get_dummies(flow['视频编号'])
账号_dum=pd.get_dummies(flow['账号'])
flow = pd.concat([flow,推广位名称_dum,品牌名称_dum,视频编号_dum,账号_dum],axis=1)

In [28]:
# 缺失值复杂填充：因果关系预测
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import Ridge
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

In [29]:
# 拆分用于数据填充的数据集
missing_data = flow.drop(['推广位名称','品牌名称','视频编号','账号'],axis=1)

In [11]:
# BayesianRidge填充:效果不如trees好，出现了很多异常值
# brimputer = IterativeImputer(estimator=BayesianRidge(), n_nearest_features=None, imputation_order='ascending')# 定义估算器
# brimputer_imputed = pd.DataFrame(brimputer.fit_transform(missing_data), columns=missing_data.columns)# fit and transform

In [30]:
# Trees填充
etimputer = IterativeImputer(estimator=ExtraTreesRegressor(n_estimators=10, random_state=0))
etimputer_imputed = pd.DataFrame(etimputer.fit_transform(missing_data), columns=missing_data.columns)

C:\Users\Sunny\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Sunny\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):


In [31]:
pd.isnull(etimputer_imputed.iloc[:,:23]).sum()

已投放天数          0
投放时段           0
消耗             0
展示数            0
点击数            0
转化数            0
按钮button       0
调起店铺           0
播放数            0
有效播放数          0
WiFi播放占比(%)    0
商品ID           0
页面PV           0
页面UV           0
收藏UV           0
加入购物车UV        0
综合成交笔数         0
综合成交金额         0
直接成交订单笔数       0
直接成交订单金额       0
间接成交笔数         0
间接成交金额         0
D5pro1         0
dtype: int64

In [32]:
flow1 = etimputer_imputed.copy()

In [33]:
# 特征创造
# 标签（业绩表现）
flow1['点击ROI']=flow1['直接成交订单金额']/flow1['消耗']
flow1['综合ROI']=flow1['综合成交金额']/flow1['消耗']
flow1['直接成交率']=flow1['直接成交订单笔数']/flow1['页面UV']
flow1['综合成交率']=flow1['综合成交笔数']/flow1['页面UV']
# 特征
flow1['有效播放率'] = flow1['有效播放数']/flow1['播放数']
flow1['点击率'] = flow1['点击数']/flow1['播放数']
flow1['转化率'] = flow1['页面UV']/flow1['点击数']
flow1['收藏率']=flow1['收藏UV']/flow1['页面UV']
flow1['加购率']=flow1['加入购物车UV']/flow1['页面UV']

In [34]:
# 转化数据类型
flow1['直接成交订单笔数'] = flow1['直接成交订单笔数'].astype('int64')

In [35]:
# 填充新建特征空值
pd.isnull(flow1.iloc[:,-10:]).sum()
flow1.replace([np.inf, -np.inf], np.nan, inplace=True)
flow1.fillna(value=0,inplace=True)

In [36]:
flow = pd.concat([flow1,flow['品牌名称']],axis=1)

In [68]:
# 正态性检验: ks H0:点击ROI服从正态分布
from scipy.stats import kstest
test_stat = kstest(flow['点击ROI'], 'norm')

KstestResult(statistic=0.5, pvalue=0.0)

In [ ]:
# 输出数据备份
flow.to_csv('信息流数据_v2.csv',index=False)